# WIP: Back-test

This example is still WIP.
* Load Data
* Back Test - the Basic Idea

In [1]:
from grandma_stock_valuation import FileLogger, loadPacakgeData
from grandma_stock_valuation import batchValuation, addCashPortfolio, getCorrelationWeight, allocatePortfolio

# Refer to example_0_FileLogger.ipynb for details of the FileLogger.
logger = FileLogger()
logPrint = logger.logPandas

### Load Data

For this example, I will use the stored package data.

To query data from Yahoo, please refer to *example_1_yahoo_data_loader.ipynb*.

In [2]:
d_instrument_data, d_instrument = loadPacakgeData(verbose=2)

d_instrument

EEMA data contains 2530 rows, 2530 dates from 2012-02-09 to 2022-02-28.
IEV data contains 5431 rows, 5431 dates from 2000-07-28 to 2022-02-28.
IVV data contains 5479 rows, 5479 dates from 2000-05-19 to 2022-02-28.
VPL data contains 4273 rows, 4273 dates from 2005-03-10 to 2022-02-28.


{'IVV': 'SP500',
 'VPL': 'Developed Asia-Pacific',
 'IEV': 'Europe',
 'EEMA': 'Emerging Asia'}

In [3]:
logPrint("Keys of d_instrument_data:", str(d_instrument_data.keys()))

logPrint("IVV (SP500 ETF):", d_instrument_data['IVV'].head())

2022-03-06 07:25:00,109 INFO Keys of d_instrument_data: dict_keys(['EEMA', 'IEV', 'IVV', 'VPL'])
2022-03-06 07:25:00,114 INFO IVV (SP500 ETF): 
        date       open       high        low      close  close_adj   volume
0 2000-05-19  142.65625  142.65625  140.25000  140.68750  94.121216   775500
1 2000-05-22  140.59375  140.59375  136.81250  139.81250  93.535789  1850600
2 2000-05-23  140.21875  140.21875  137.68750  137.68750  92.114151   373900
3 2000-05-24  137.75000  140.06250  136.65625  139.75000  93.494003   400300
4 2000-05-25  140.03125  140.93750  137.87500  138.46875  92.636810    69600



### Back Test - the Basic Idea

WIP
* back test is to understand how your strategy behaves under various scenanios, not to prove your strategy can beat xxx.
* most investment products are not built to beat xxx.
* historical performance cannot represent future performance.


### Back Test - WIP

In [53]:
price_col = 'close_adj'
end_date = None # '2022-01-31'
backtest_years = 10
minimum_training_years = 5
maximum_training_years = 10
adjust_months = 1

with_cash = True
with_correlation_weights = True
verbose = 2
printfunc = print

import pandas as pd


In [38]:
# clean price data, get start and end date of each ticker
d_instrument_prices = {}
d_instrument_start_dates = {}
d_instrument_end_dates = {}
for ticker, df_price in d_instrument_data.items():
    df_price = df_price[df_price[price_col] > 0][['date', price_col]].reset_index(drop=True)
    df_price['date'] = pd.to_datetime(df_price['date'])
    d_instrument_prices[ticker] = df_price
    d_instrument_start_dates[ticker] = df_price['date'].min()
    d_instrument_end_dates[ticker] = df_price['date'].max()

# dates of back-test periods
if end_date is None:
    backtest_end_date = max(d_instrument_end_dates.values())
else:
    backtest_end_date = pd.to_datetime(end_date)

backtest_start_date = backtest_end_date - pd.DateOffset(years=backtest_years)
if verbose > 0: printfunc(f"To backtest {backtest_years} years, from {backtest_start_date.date()} to {backtest_end_date.date()}")

at_least_start_date = backtest_start_date - pd.DateOffset(years=minimum_training_years)
at_most_start_date = backtest_start_date - pd.DateOffset(years=maximum_training_years)

for ticker, d in d_instrument_start_dates.items():
    if d > at_least_start_date:
        printfunc(f"{ticker}'s start date {d.date()} is beyond {at_least_start_date.date()} for full back-test.")

# combine cleaned price data into one dateframe
df_instrument_prices = pd.DataFrame()
for ticker, df_price in d_instrument_prices.items():
    df = df_price[df_price['date']>=at_most_start_date].copy()
    df.rename(columns={price_col:'price_'+ticker}, inplace=True)

    if len(df_instrument_prices)==0:
        df_instrument_prices = df.copy()
        continue
    else:
        df_instrument_prices = df_instrument_prices.merge(df, 'outer', 'date')

df_instrument_prices = df_instrument_prices.sort_values('date').reset_index(drop=True)
df_instrument_prices.fillna(method='pad', inplace=True)

df_instrument_prices.index = df_instrument_prices['date']
df_instrument_prices.drop(columns='date', inplace=True)

del d_instrument_prices

df_instrument_prices.head()


To backtest 10 years, from 2012-02-28 to 2022-02-28
EEMA's start date 2012-02-09 is beyond 2007-02-28 for full back-test.


,price_EEMA,price_IEV,price_IVV,price_VPL
date,,,,
2002-02-28,NaN,16.412029,75.591682,NaN
2002-03-01,NaN,16.572498,77.443367,NaN
2002-03-04,NaN,17.118109,78.743614,NaN
2002-03-05,NaN,16.992653,78.559822,NaN
2002-03-06,NaN,17.223148,79.465218,NaN


In [39]:
df_prices_i = df_instrument_prices.loc[:backtest_start_date]

df_prices_i.tail()

,price_EEMA,price_IEV,price_IVV,price_VPL
date,,,,
2012-02-22,45.725971,27.821569,112.755722,40.177986
2012-02-23,45.825390,28.091753,113.292702,40.376316
2012-02-24,45.667973,28.331923,113.482681,40.460224
2012-02-27,45.146011,28.121778,113.730515,40.361053
2012-02-28,45.518845,28.361938,114.044487,40.750103


In [46]:
d_instrument_data_i = {}
for col in df_prices_i.columns:
    ticker = col.replace('price_','')
    d_instrument_data_i[ticker] = df_prices_i[col].reset_index().rename(columns={col:'price'})

d_instrument_data_i.keys()

dict_keys(['EEMA', 'IEV', 'IVV', 'VPL'])

In [59]:
total_dollar = 100

df_metrics_i, _ = batchValuation(
    d_instrument_data_i,
    init_parameters={'recent_months':0, 'train_years':10, 'date_end':None}, # min_train_years
    fit_parameters={'price_col':'price', 'log':True, 'n_std':1.5},
    valuate_parameters={'min_annual_return':0.01},
    save_result=False,
    metric_file = None,
    figure_folder = None,
    verbose=0,
    printfunc=logPrint
)

if with_cash:
    df_metrics_i = addCashPortfolio(df_metrics_i)

if with_correlation_weights:
    cor_weights = getCorrelationWeight(d_instrument_data_i, price_col='price', with_cash=with_cash, verbose=0, printfunc=printfunc)

df_metrics_i['weight'] = df_metrics_i['ticker'].apply(lambda t: cor_weights[t])

df_metrics_i['portfolio_allocation'] = allocatePortfolio(
    df_metrics_i['over_value_years'],
    transformation='exponential',
    scale=1,
    with_cash=True,
    weights=df_metrics_i['weight']
)

df_metrics_i['dollar'] = total_dollar * df_metrics_i['portfolio_allocation']

df_metrics_i


,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years,weight,portfolio_allocation,dollar
0,EEMA,0.337642,0.052055,0.471226,45.518845,45.576127,-0.001257,-0.059226,0.351880,NaN,NaN
1,IEV,0.417333,10.005479,0.059537,28.361938,31.373958,-0.096004,-0.571574,0.141872,NaN,NaN
2,IVV,0.505924,10.005479,0.040049,114.044487,107.405568,0.061812,1.543402,0.146987,NaN,NaN
3,VPL,-0.001439,6.975342,-0.001291,40.750103,38.901705,0.047515,NaN,0.159261,NaN,NaN
4,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.200000,NaN,NaN


In [ ]:
def updatePrice(df_portfolio, d, d_data=d_data, price_col=price_col):
    l_current_price = []
    for name, data in d_data.items():
        df = data[data['date']<=d].copy()
        l_current_price.append(pd.DataFrame({'name':[name], 'price':df[price_col].iloc[-1]}))
    df_current_price = pd.concat(l_current_price).reset_index(drop=True)

    df_portfolio = df_portfolio.merge(df_current_price, 'left', 'name')
    df_portfolio['dollar'] = df_portfolio['dollar'] * df_portfolio['price'] / df_portfolio['currenct_price']
    df_portfolio['portfolio_allocation'] = df_portfolio['dollar'] / df_portfolio['dollar'].sum()
    df_portfolio['currenct_price'] = df_portfolio['price']
    df_portfolio.drop(columns=['price'], inplace=True)

    return df_portfolio


d_portfolio = {}
d_total_dollar = {}
rebalance_date = backtest_start_date + pd.DateOffset(months=1)

for d in pd.date_range(backtest_start_date, backtest_end_date):
    df_portfolio = updatePrice(df_portfolio, d)
    total_dollar = df_portfolio['dollar'].sum()

    if d == rebalance_date:
        total_dollar = df_portfolio['dollar'].sum()
        df_portfolio = getAllocation(total_dollar=total_dollar, d_end_date=d)
        rebalance_date = rebalance_date + pd.DateOffset(months=1)
        print(f"Rebalance on {d.date()}, total dollar = {total_dollar}")

    d_portfolio[d] = df_portfolio
    d_total_dollar[d] = total_dollar

print(f"final portfolio increased by {total_dollar/100-1:.3f} over {backtest_years} years, which is {(total_dollar/100)**(1/backtest_years)-1:.4f} annualized growth.")